In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '../../')

In [2]:
import torch
from src.models.ecg_step_module import EcgEmbed
from src.basic.dx_and_feat import Feature, zero_vec

N = 64 # batch_size
batched_ecg = torch.rand(N, 12, 5000)
batched_feat = zero_vec(Feature, [N])
batched_input = (batched_ecg, batched_feat)
all_mid_output = {'pipeline': {}}
embed = EcgEmbed(all_mid_output)(batched_input)
print(all_mid_output['EcgEmbed']['embed'].shape)

torch.Size([64, 240])


In [3]:
# from torch import nn
# from pmlayer.torch.hierarchical_lattice_layer import HLattice

# output_layer = nn.Linear(64, 1)
# consequent_model = nn.Sequential(self.embed_layer, output_layer)
# HLattice(263, sizes, self.lattice_inc_indices, consequent_model)

In [4]:
from src.models.ecg_step_module import BlockModule
# all_mid_output['BlockModule'].clear()
block_module = BlockModule(all_mid_output)
# print(block_module)
block_module(batched_input)

{'feat': tensor(0., grad_fn=<BinaryCrossEntropyBackward0>),
 'delta': tensor([0.], grad_fn=<AddBackward0>)}

In [5]:
print(all_mid_output['BlockModule']['LPR_imp'].shape)
print(all_mid_output['BlockModule']['LQRS_imp'].shape)
print(all_mid_output['BlockModule']['AVB'].shape)
print(all_mid_output['BlockModule']['LBBB'].shape)
print(all_mid_output['BlockModule']['RBBB'].shape)

torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])


In [23]:
from src.basic.rule_ml import SeqSteps
from src.models.ecg_step_module import EcgEmbed, BlockModule
from src.basic.dx_and_feat import Feature, zero_vec

N = 64 # batch_size
batched_ecg = torch.rand(N, 12, 5000)
batched_feat = zero_vec(Feature, [N])
batched_input = (batched_ecg, batched_feat)
all_mid_output = {}

# test SeqSteps
seq1 = SeqSteps('seq1', all_mid_output, [EcgEmbed(all_mid_output), BlockModule(all_mid_output)])
losses = seq1(batched_input)
print(losses)

<class 'dict'>
<class 'dict'>
<class 'dict'>
(0, 0)


In [34]:
from src.basic.rule_ml import PipelineModule
from src.basic.dx_and_feat import Diagnosis
y = torch.zeros([N, len(Diagnosis)], dtype=torch.float32)
batch = (batched_input, y)
# test PipelineModule
pipeline_module = PipelineModule()
all_mid_output = pipeline_module.all_mid_output
seq2 = SeqSteps('seq2', all_mid_output, [EcgEmbed(all_mid_output), BlockModule(all_mid_output)])
pipeline_module.add_pipeline(seq2)
losses = pipeline_module.training_step(batch, 0)
print(losses)

tensor(0.1047, grad_fn=<AddBackward0>)


## Test Trainer

In [ ]:

# training params
max_epochs = 50
save_top_k = 5

# Define hyperparameters here

# model with suggested hyperparameters
pipeline_module = get_default_pipeline()
trainer = Trainer(
    callbacks=get_trainer_callbacks(trial, save_top_k),
    #   limit_train_batches=4,
    #   limit_val_batches=4,
    #   limit_test_batches=4,
    max_epochs=max_epochs,
    accelerator="auto",
    logger=pl_loggers.TensorBoardLogger(save_dir=save_dir),
    log_every_n_steps=LOG_INTERVAL,
    check_val_every_n_epoch=CHECK_VAL_EVERY_N_EPOCH,
    enable_progress_bar=False)

# record hyperparameters
# hyperparams = dict(num_of_naive_net_layers=num_of_naive_net_layers, naive_net_output_dims=naive_net_output_dims)
# trainer.logger.log_hyperparams(hyperparams)
trainer.fit(pipeline_module, data_module)

return trainer.callback_metrics["val_epoch/total_loss"].item()

## Test Trained Model

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '../../')

In [2]:
import os
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from src.models.ecg_step_module import EcgPipeline
from src.utils.data_utils import EcgDataModule
from src.utils.train_utils import get_common_trainer_params, set_cuda_env
from src.basic.constants import BEST_MODEL_PATH

save_dir = BEST_MODEL_PATH
ckpt_name = 'MPAV_rho8-auroc=0.8360216.ckpt'
ckpt_path = os.path.join(save_dir, ckpt_name)

model = EcgPipeline.load_from_checkpoint(ckpt_path)

In [3]:
agg_path = os.path.join(save_dir, 'MPAV_rho8_val_mid_output_agg.csv')
model.generate_report(agg_path, 9)